<h1 align=center>King County Housing Analysis</h1>


### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from math import sqrt
from random import gauss
from mpl_toolkits import mplot3d
import sklearn.metrics as metrics
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')



 <h2 align=center>Understanding the Data</h2>

In [2]:
# IMPORTING THE DATA 
df = pd.read_csv('../data/kc_house_data.csv')


In [3]:
# SHAPE OF DATA FRAME
df.shape

(21597, 21)

In [4]:
# DATA FRAME
df.head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
5,7237550310,5/12/2014,1230000.0,4,4.50,5420,101930,1.0,0.0,0.0,...,11,3890,1530.0,2001,0.0,98053,47.6561,-122.005,4760,101930
6,1321400060,6/27/2014,257500.0,3,2.25,1715,6819,2.0,0.0,0.0,...,7,1715,?,1995,0.0,98003,47.3097,-122.327,2238,6819
7,2008000270,1/15/2015,291850.0,3,1.50,1060,9711,1.0,0.0,NaN,...,7,1060,0.0,1963,0.0,98198,47.4095,-122.315,1650,9711
8,2414600126,4/15/2015,229500.0,3,1.00,1780,7470,1.0,0.0,0.0,...,7,1050,730.0,1960,0.0,98146,47.5123,-122.337,1780,8113
9,3793500160,3/12/2015,323000.0,3,2.50,1890,6560,2.0,0.0,0.0,...,7,1890,0.0,2003,0.0,98038,47.3684,-122.031,2390,7570


In [5]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [12]:
# UNIQUE YEAR FOR THIS DATA FRAME

df['year'] = pd.DatetimeIndex(df['date']).year
df['year'].unique()

array([2014, 2015])

In [ ]:
pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if int(x) == x else '{:,.2f}'.format(x)
df.describe()

In [ ]:
df.info()

In [ ]:
# PLOT DATA HISTOGRAM
df.hist(figsize=(15,15), edgecolor = 'black');

In [ ]:
# HEATMAP FOR CORRELATION DATAFRAME

corr = df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(15, 20))

sns.heatmap(corr, mask=mask, cmap='coolwarm', vmax=1, center=0,
            square=True, linewidths=.5,annot=True, cbar_kws={"shrink": .5});

 <h2 align=center>Data Cleaning</h2>

**Check List:**

* Missing values
* Data type conversions
* Checking for and removing multicollinearity
* Normalizing our numeric data
* Converting categorical data to numeric



In [ ]:
# CHECKING FOR DUPLICATES

len(set(df['id']))

In [ ]:
# DROPPING DUPLICATES IN ID COLUMN

df.drop_duplicates(subset='id', inplace=True)
df.shape

In [ ]:
# CHECKING FOR OBJECTS IN SQFT_BASEMENT

df['sqft_basement'].head(10)

In [ ]:
# REPLACE VALUES IN 'SQFT_BASEMENT' == TO '?' FOR '0'

df.loc[df['sqft_basement'] == '?','sqft_basement'] = 0.0

In [ ]:
# CHECKING 'YR_RENOVATED' 

df['yr_renovated'].head()

In [ ]:
# CHANGING 'YR_RENOVATED' FOR 'INT64'

df['yr_renovated'] = df['yr_renovated'].astype('Int64')
df['yr_renovated']

In [ ]:
# CHECK FOR NULL VALUES

df.isna().sum()

In [ ]:
# 'WATERFRONT' VALUES

df['waterfront'].unique()

In [ ]:
# 'VIEW' VALUES

df['view'].unique()

In [ ]:
# 'GRADE' VALUES

df['grade'].unique()

In [ ]:
# 'condition' VALUES

df['condition'].unique()

In [ ]:
# FILL NAN IN 'YR_RENOVATED' WITH '0'

df['yr_renovated'] = df['yr_renovated'].fillna(0)

In [ ]:
df.columns

In [ ]:
# DROPPING COLUMNS

df.drop(columns = ['id', 'date', 'view', 'sqft_above', 'sqft_basement', 'yr_renovated', 
                   'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15'], inplace = True)

In [ ]:
df

In [ ]:
df.corr()

In [ ]:
# HEATMAP FOR CORRELATION DATAFRAME

corr = df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(15, 20))

sns.heatmap(corr, mask=mask, cmap='coolwarm', vmax=1, center=0,
            square=True, linewidths=.5,annot=True, cbar_kws={"shrink": .5});

In [ ]:
df.columns

In [ ]:
# PLOTING RELATIONSHIP BETWEEN PRICE (BEDROOM, BATHROOM AND SQFT_LINVING)

fig, axs = plt.subplots(1, 3, sharey=True, figsize=(18, 6))
for idx, channel in enumerate(['bedrooms', 'bathrooms', 'sqft_living']):
    df.plot(kind='scatter', x=channel, y='price', ax=axs[idx], label=channel)
plt.legend()
plt.show()

In [ ]:
# PLOTING RELATIONSHIP BETWEEN PRICE (SQFT_LOT, FLOORS AND WATERFRONT)

fig, axs = plt.subplots(1, 3, sharey=True, figsize=(18, 6))
for idx, channel in enumerate(['sqft_lot', 'floors', 'waterfront']):
    df.plot(kind='scatter', x=channel, y='price', ax=axs[idx], label=channel)
plt.legend()
plt.show()

In [ ]:
# PLOTING RELATIONSHIP BETWEEN PRICE (CONDITION, GRADE AND YR_BUILT)

fig, axs = plt.subplots(1, 3, sharey=True, figsize=(18, 6))
for idx, channel in enumerate(['condition', 'grade', 'yr_built']):
    df.plot(kind='scatter', x=channel, y='price', ax=axs[idx], label=channel)
plt.legend()
plt.show()

In [ ]:
# DROPPING 'WATERFRONT'

df.drop(columns = ['waterfront'], inplace = True)

In [ ]:
# WE CAN SEE 33 BEDROOMS AND OTHER ANOMALIES THAT WILL BE CONSIDER OUTLIERS

pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if int(x) == x else '{:,.2f}'.format(x)
df.describe()

In [ ]:
# OBTAINING Z-SCORES  
zscore = np.abs(stats.zscore(df))
zscore 

In [ ]:
# DETECTING OUTLIERS WITH OVER 3 STANDARD DEVIATION FROM THE MEAN IN DATAFRAME

outliers = df[(np.abs(zscore) > 3).any(1)]
outliers

In [ ]:
# OUTLIER DESCRIBE TABLE EXAMPLE: 33 BEDROOMS 

pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if int(x) == x else '{:,.2f}'.format(x)
outliers.describe()

In [ ]:
# OBTAINING ALL DATA THAT IS IN BETWEEN 3 STANDAR DEVIATION OF THE MEAN, ELIMINATING OUTLIERS

df = df[(zscore < 3).all(axis=1)]
df

In [ ]:
# NORMALIZING OUR NUMERIC DATA FOR DESCRIBE FUNCTION

pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if int(x) == x else '{:,.2f}'.format(x)
df.describe()

In [ ]:
# RE-PLOTING RELATIONSHIP BETWEEN PRICE (BEDROOM, BATHROOM AND SQFT_LINVING) AFTER ELIMINATING OUTLIERS

fig, axs = plt.subplots(1, 3, sharey=True, figsize=(18, 6))
for idx, channel in enumerate(['bedrooms', 'bathrooms', 'sqft_living']):
    df.plot(kind='scatter', x=channel, y='price', ax=axs[idx], label=channel)
plt.legend()
plt.show()

In [ ]:
# RE-PLOTING RELATIONSHIP BETWEEN PRICE (SQFT_LOT AND FLOORS) AFTER ELIMINATING OUTLIERS

fig, axs = plt.subplots(1, 2, sharey=True, figsize=(18, 6))
for idx, channel in enumerate(['sqft_lot', 'floors']):
    df.plot(kind='scatter', x=channel, y='price', ax=axs[idx], label=channel)
plt.legend()
plt.show()

In [ ]:
# RE-PLOTING RELATIONSHIP BETWEEN PRICE (CONDITION, GRADE AND YR_BUILT) AFTER ELIMINATING OUTLIERS


fig, axs = plt.subplots(1, 3, sharey=True, figsize=(18, 6))
for idx, channel in enumerate(['condition', 'grade', 'yr_built']):
    df.plot(kind='scatter', x=channel, y='price', ax=axs[idx], label=channel)
plt.legend()
plt.show()

 <h2 align=center>Baseline Model</h2>

In [ ]:
# DROPPING COLUMNS
df.drop(columns = ['floors', 'yr_built'], inplace = True)

In [ ]:
df.columns

In [ ]:
# DATA SEPARATION INTO PREDITORS AND TARGET

house_preds = df.drop('price', axis=1)
house_target = df['price']
house_preds.head()

In [ ]:
# INDIPENDENT VARIABLES 

predictors = sm.add_constant(house_preds)
predictors.head()

In [ ]:
# BASELINE MODEL 

baseline_model = sm.OLS(house_target, predictors).fit ()


In [ ]:
baseline_model.summary()


**This means that 59.5% of the variation in the dependent variable is explained by variation in the explanatory variable. Meaning that we still have 40.5% of the variation is unexplained by the model.** 

**So let's get back to it to refine our model.**

In [ ]:
residuals = baseline_model.resid
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True, )    
plt.show;

# Assumption of regression

In [ ]:
# Correlation 'Price' vs all variables

data_corrs = df.corr()['price'].map(abs).sort_values(ascending=False)
data_corrs

In [ ]:
df.columns

In [ ]:
# PLOTING RELATIONSHIP BETWEEN PRICE (BEDROOM, BATHROOM, SQFT_LINVING AND GRADE) AFTER ELIMINATING OUTLIERS

fig, axs = plt.subplots(1, 4, sharey=True, figsize=(18, 6))
for idx, channel in enumerate(['bedrooms', 'bathrooms', 'sqft_living', 'grade']):
    df.plot(kind='scatter', x=channel, y='price', ax=axs[idx], label=channel)
plt.legend()
plt.show()

In [ ]:
# Regression Plots for 'Price vs Sqft_living'

fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(baseline_model, "sqft_living", fig=fig)
plt.show()

Now we see that is a strong correlation meaning that has a homoscedasticity between price and sqft_living is following a great line. Also we can see that bedrooms, bathrooms and grade are categorical so we will need to create dummy variables for each so we can have a more accurate model.

In [ ]:
# PREPARATION OF BATHROOMS COLUMNS

df = df[['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'condition', 'grade']] # deleted foors and yr_built
df.loc[df['bathrooms'] <= 1,'bathrooms'] = 1
df.loc[(df['bathrooms'] > 1) & (df['bathrooms'] <= 2),'bathrooms'] = 2
df.loc[(df['bathrooms'] > 2) & (df['bathrooms'] <= 3),'bathrooms'] = 3
df.loc[(df['bathrooms'] > 3) & (df['bathrooms'] <= 4),'bathrooms'] = 4
df.loc[(df['bathrooms'] > 4) & (df['bathrooms'] <= 5),'bathrooms'] = 5
df.loc[(df['bathrooms'] > 5) & (df['bathrooms'] <= 6),'bathrooms'] = 6
df.loc[(df['bathrooms'] > 6) & (df['bathrooms'] <= 7),'bathrooms'] = 7
df.loc[(df['bathrooms'] > 7) & (df['bathrooms'] <= 8),'bathrooms'] = 8

In [ ]:
# CREATING A COPY OF DATA FRAME BEFORE MODIFIYING WITH DUMMIES FOR FUTURE MODELS

df_model = df.copy()
df_model

In [ ]:
pd.plotting.scatter_matrix(df, figsize=(18,12));


In [ ]:
df.columns

In [ ]:
df

In [ ]:
# CHANGING 'BATHROOMS' TO INT64

df['bathrooms'] = df['bathrooms'].astype('Int64')

 <h3 align=center>Data with Dummies</h3>

In [ ]:
# ADDING DUMMIES FOR "GRADE", "BEDROOMS" AND "BATHROOMS" 

df_dummies = pd.get_dummies(df, columns=['grade', 'bedrooms', 'bathrooms'])

In [ ]:
df.info()

In [ ]:
#df.columns

df_dummies.columns

# Model 2 

In [ ]:
outcome = 'price'
x_cols = ['sqft_living', 'sqft_lot', 'condition',  # yr_built and # floors
       'grade_5', 'grade_6', 'grade_7', 'grade_8', 'grade_9', 'grade_10',
       'grade_11', 'bedrooms_1', 'bedrooms_2', 'bedrooms_3', 'bedrooms_4',
       'bedrooms_5', 'bedrooms_6', 'bathrooms_1', 'bathrooms_2', 'bathrooms_3',
       'bathrooms_4', 'bathrooms_5']
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors
model_2 = ols(formula=formula, data=df_dummies).fit()


model_2.summary()

In [ ]:
residuals = model_2.resid
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True, )    
plt.show;

 <h3 align=center>Validation Model 2</h3>

In [ ]:
# SELECTING 'X' AND 'Y' VALUES FROM 'DATA_LOG' FOR TRAIN-TEST SPLIT
X = df_dummies.drop('price', axis=1)
y = df_dummies['price']

df_dummies

In [ ]:
# TRAIN-TEST SPLIT0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
print(len(X_train), len(X_test), len(y_train), len(y_test))

In [ ]:
# APPLY AND MODEL THE TRAIN-TEST SET
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)


#CALCULATE TRAINING AND TEST RESIDUALS
train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test


# CALCUALTE MEAN SQUARE ERROR
train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squared Error:', train_mse)
print('Test Mean Squared Error:', test_mse)

In [ ]:
# CROSS VALIDATION RESULTS 
mse = make_scorer(mean_squared_error)
cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring=mse)
cv_5_results.mean()

In [ ]:
# PLOT DATA
df_dummies.hist(column= ['price', 'sqft_living'] ,figsize=(8,3), edgecolor = 'black');

<h3 align=center>Data Log Transformation</h3>

In [ ]:
# PERFORM DATA LOG TRANSFORMATION 

data_log = pd.DataFrame([])
   
data_log['price_log'] = np.log(df_model['price'])
data_log['bedrooms'] = (df_model['bedrooms'])
data_log['bathrooms'] = (df_model['bathrooms'])
data_log['sqft_living_log'] = np.log(df_model['sqft_living'])
data_log['sqft_lot_log'] = np.log(df_model['sqft_lot'])
data_log['condition'] = (df_model['condition'])
data_log['grade'] = (df_model['grade'])
   
data_log.hist(figsize  = [18,8]);

In [ ]:
# CHANGING 'BATHROOMS' TO INT64

data_log['bathrooms'] = data_log['bathrooms'].astype('Int64')

In [ ]:
# ADDING DUMMIES FOR "GRADE", "BEDROOMS" AND "BATHROOMS" 

data_log = pd.get_dummies(data_log, columns=['grade', 'bedrooms', 'bathrooms', 'condition'])

In [ ]:
data_log

# Model 3

In [ ]:
data_log.columns

In [ ]:
outcome = 'price_log'
x_cols = ['sqft_living_log', 'sqft_lot_log', 'grade_5',
       'grade_6', 'grade_7', 'grade_8', 'grade_9', 'grade_10', 'grade_11',
       'bedrooms_1', 'bedrooms_2', 'bedrooms_3', 'bedrooms_4', 'bedrooms_5',
       'bedrooms_6', 'bathrooms_1', 'bathrooms_2', 'bathrooms_3',
       'bathrooms_4', 'bathrooms_5', 'condition_2', 'condition_3', 'condition_4',
       'condition_5']
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors
model_3 = ols(formula=formula, data=data_log).fit()

model_3.summary()

In [ ]:
 residuals = model_3.resid
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True, )    
plt.show;

 <h3 align=center>Validation Model 3</h3>

In [ ]:
# SELECTING 'X' AND 'Y' VALUES FROM 'DATA_LOG' FOR TRAIN-TEST SPLIT

X = data_log.drop('price_log', axis=1)
y = data_log['price_log']

data_log

In [ ]:
# TRAIN-TEST SPLIT

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
print(len(X_train), len(X_test), len(y_train), len(y_test))

In [ ]:
# APPLY AND MODEL THE TRAIN-TEST SET
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)


#CALCULATE TRAINING AND TEST RESIDUALS
train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test


# CALCUALTE MEAN SQUARE ERROR
train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squared Error:', train_mse)
print('Test Mean Squared Error:', test_mse)

In [ ]:
# CROSS VALIDATION RESULTS 
mse = make_scorer(mean_squared_error)
cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring=mse)
cv_5_results.mean()